In [5]:
!pip uninstall keras -y
!pip install keras==2.3.1

Found existing installation: Keras 2.3.1
Uninstalling Keras-2.3.1:
  Successfully uninstalled Keras-2.3.1
  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [6]:
import keras 
from keras.models import load_model
from keras.layers import (Input, Concatenate, Dense, Lambda, Reshape, Flatten, Activation, Conv2D, RepeatVector)
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Deconv2D
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.utils import to_categorical
from sklearn.utils import shuffle
from keras.losses import binary_crossentropy
import tensorflow as tf
import numpy
import numpy as np
from sklearn.model_selection import train_test_split
import h5py

In [7]:
# load jet images and labels
data = h5py.File('../input/jets-dataset/PYTHIA_JETS.hdf5', 'r')
JETS = numpy.array(data["image"][:])

data2 = h5py.File('../input/jets-dataset/PYTHIA_JETS.hdf5', 'r')
JETS_label = numpy.array(data2["signal"][:])

JETS, JETS_label = shuffle(JETS, JETS_label)

JETS_label = JETS_label[0:870000].reshape(870000,1)
JETS = JETS[0:870000].reshape(870000,25,25,1)

JETS[JETS <1e-3] = 0

In [8]:
# load feature perceptual classifier and select layers
classifier = load_model('../input/classifier/11X11_CNN_LR.h5')
selected_layers = ['leaky_re_lu_1','leaky_re_lu_2']

# define weights for each layer in the feature perceptual loss
selected_weights = [1, 1]

In [9]:
# split data into training and validation 
X_train, X_test, Y_train, Y_test = train_test_split(JETS, JETS_label, train_size=0.9)

# normalize images to gray scale (0-1)
X_train = X_train.astype('float32')/100.
X_test = X_test.astype('float32')/100.

# create categorical classes of labels
Y_train = to_categorical(Y_train, num_classes = 2)
Y_test = to_categorical(Y_test, num_classes = 2)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(783000, 25, 25, 1) (783000, 2)
(87000, 25, 25, 1) (87000, 2)


In [10]:
# define basic variables such as latent size, batch size, label size, and number of epochs
latent_size = 12
batch_size = 100
c_space = Y_train.shape[1]
n_epochs = 12

In [11]:
# define image and label shape
image = Input(shape = (25,25,1))
label = Input(shape = (Y_train.shape[1],))

# tranform labels into 25x25x2 images and combine with input image
transform = RepeatVector(625)(label)
label_2D = Reshape((25,25, Y_train.shape[1]))(transform)
inputs = Concatenate()([image, label_2D])

x = Conv2D(32, kernel_size = (3,3), padding = 'same')(inputs)
x = LeakyReLU()(x)

x = Conv2D(64, kernel_size = (3,3), padding = 'same', strides = 2)(x)
x = LeakyReLU()(x)

x = Conv2D(128, kernel_size = (3,3), padding = 'same', strides = 2)(x)
x = LeakyReLU()(x)

encoder_output = Flatten()(x)

mu = Dense(latent_size, activation = 'linear')(encoder_output)
sigma = Dense(latent_size, activation = 'linear')(encoder_output)

# sampling layer
def sampling(args):
    mu, sigma = args
    eps = K.random_normal(shape=(batch_size, latent_size), mean=0., stddev=1.0)
    return mu + K.exp(sigma/2.) * eps

latent_space = Lambda(sampling, output_shape=(latent_size,))([mu, sigma])

# combine label to latent space
combined_space = Concatenate()([latent_space, label])

decoder1 = Dense((7*7*128))
decoder2 = Reshape((7,7,128))

decoder3 = Deconv2D(64, kernel_size = (3, 3), padding='same', strides = 2)
decoder4 = LeakyReLU()

decoder5 = Deconv2D(32, kernel_size = (3, 3), padding='same', strides = 2)
decoder6 = LeakyReLU()

decoder7 = Conv2D(1, kernel_size = (4,4), padding='valid')

decoder_out = Activation('relu')

In [12]:
# encoder
encoder = Model(inputs = [image, label], outputs = latent_space)
encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 625, 2)       0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 25, 25, 1)    0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 25, 25, 2)    0           repeat_vector_1[0][0]            
____________________________________________________________________________________________

In [13]:
# decoder
inputs = Input(shape=(latent_size + c_space, ))

d = decoder1(inputs)
d = decoder2(d)
d = decoder3(d)
d = decoder4(d)
d = decoder5(d)
d = decoder6(d)
d = decoder7(d)
d = decoder_out(d)
decoder = Model(inputs, d)
decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 14)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6272)              94080     
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        73792     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 32)        18464     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 28, 28, 32)        0   

In [14]:
# define VAE losses 

# total loss
def DFC_loss(x_in, x_out):
    kl_loss = 0.5 * K.sum(K.exp(sigma) + K.square(mu) - 1. - sigma, axis=-1)
    return K.mean(kl_loss + feature_perceptual(x_in,x_out) + bce_loss(x_in, x_out))

# Bernoulli loss 
def bce_loss(x_in, x_out):
    x_in = K.reshape(x_in, shape=(batch_size, 625))
    x_out = K.reshape(x_out, shape=(batch_size, 625))
    
    return binary_crossentropy(x_in, x_out)

# feature perceptual loss
def feature_perceptual(x_in, x_out):
    
    x_in = K.reshape(x_in, shape=(batch_size, 25,25,1))
    x_out = K.reshape(x_out, shape=(batch_size, 25,25,1))
    
    conv_outputs = [classifier.get_layer(l).output for l in selected_layers]
    activation = Model(classifier.input, conv_outputs)
    list_1 = activation(x_in)
    list_2 = activation(x_out)
    
    for calc1, calc2, weight in zip(list_1, list_2, selected_weights):
        calc1 = K.batch_flatten(calc1)
        calc2 = K.batch_flatten(calc2)
        dfc_loss = weight * K.sum(K.square(calc1 - calc2), axis=-1)
    
    return dfc_loss

In [15]:
# VAE
d_c = decoder1(combined_space)
d_c = decoder2(d_c)
d_c = decoder3(d_c)
d_c = decoder4(d_c)
d_c = decoder5(d_c)
d_c = decoder6(d_c)
d_c = decoder7(d_c)
dc_out = decoder_out(d_c)

VAE = Model(inputs = [image, label], outputs = dc_out)
VAE.compile(optimizer = "adam", loss = DFC_loss, metrics = [feature_perceptual])
VAE.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 625, 2)       0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 25, 25, 1)    0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 25, 25, 2)    0           repeat_vector_1[0][0]            
____________________________________________________________________________________________

In [16]:
VAE_hist = VAE.fit([X_train, Y_train], X_train, verbose = 1, batch_size=batch_size, epochs=n_epochs,
                            validation_data = ([X_test, Y_test], X_test), callbacks = [EarlyStopping(patience = 5)])

Train on 783000 samples, validate on 87000 samples
Epoch 1/12
783000/783000 [==============================] - 102s 131us/step - loss: 62.0653 - feature_perceptual: 43.2174 - val_loss: 48.7052 - val_feature_perceptual: 29.2457
Epoch 2/12
783000/783000 [==============================] - 97s 123us/step - loss: 48.1041 - feature_perceptual: 29.1628 - val_loss: 46.6397 - val_feature_perceptual: 27.4947
Epoch 3/12
783000/783000 [==============================] - 97s 124us/step - loss: 46.8938 - feature_perceptual: 28.0579 - val_loss: 46.6437 - val_feature_perceptual: 27.6925
Epoch 4/12
783000/783000 [==============================] - 96s 123us/step - loss: 46.2585 - feature_perceptual: 27.4668 - val_loss: 46.3139 - val_feature_perceptual: 27.8505
Epoch 5/12
783000/783000 [==============================] - 96s 123us/step - loss: 45.8699 - feature_perceptual: 27.0995 - val_loss: 46.3854 - val_feature_perceptual: 27.2695
Epoch 6/12
783000/783000 [==============================] - 96s 123us/ste

In [17]:
VAE.save('Model4paper_VAE.h5')
decoder.save('Model4paper_decoder.h5')
encoder.save('Model4paper_encoder.h5')